In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gezinomi/miuul_gezinomi.xlsx


In [2]:
import pandas as pd
pd.set_option("display.max_rows",None)# bütün satırları görüntüler 
pd.set_option("display.float_format",lambda x:'%.2f' % x )
df=pd.read_excel('/kaggle/input/gezinomi/miuul_gezinomi.xlsx')
df.head()


,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.30,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.97,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.84,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.71,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.48,Yarım Pansiyon,İzmir,Saturday,2,Low


In [3]:
print(f'Verisetinin satır ve sütun sayısı:{df.shape}')
df.info()

Verisetinin satır ve sütun sayısı:(59164, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB


In [4]:
df["SaleCityName"].nunique()

6

In [5]:
df["SaleCityName"].value_counts()

SaleCityName
Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: count, dtype: int64

In [6]:
df["ConceptName"].nunique()


3

In [7]:
df["ConceptName"].value_counts()#hangi konseptten kaç satış gerçekleşmiş

ConceptName
Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: count, dtype: int64

In [8]:
df.groupby("SaleCityName").agg({"Price":"sum"})#ŞEHİRLERE GÖRE KAZANÇ

,Price
SaleCityName,
Antalya,2041911.10
Aydın,573296.01
Diğer,154572.29
Girne,27065.03
Muğla,665842.21
İzmir,165934.83


In [9]:
df.groupby("ConceptName").agg({"Price":"sum"})#KONSEPTLERE GÖRE KAZANÇ

,Price
ConceptName,
Herşey Dahil,3332910.77
Oda + Kahvaltı,121308.35
Yarım Pansiyon,174402.35


In [10]:
df.groupby("SaleCityName")["Price"].mean()#şehirlere göre fiyat ortalamaları

SaleCityName
Antalya   64.52
Aydın     53.86
Diğer     47.71
Girne     59.48
Muğla     62.46
İzmir     66.27
Name: Price, dtype: float64

In [11]:
df.groupby(["SaleCityName","ConceptName"])["Price"].mean()#şehir ve konsept kırılımına göre fiyat ortalamaları

SaleCityName  ConceptName   
Antalya       Herşey Dahil     64.52
              Oda + Kahvaltı   63.50
              Yarım Pansiyon   67.19
Aydın         Herşey Dahil     54.00
              Oda + Kahvaltı   34.46
              Yarım Pansiyon   30.02
Diğer         Herşey Dahil     84.77
              Oda + Kahvaltı   37.60
              Yarım Pansiyon   42.11
Girne         Herşey Dahil     97.68
              Oda + Kahvaltı   39.78
              Yarım Pansiyon   53.25
Muğla         Herşey Dahil     63.02
              Oda + Kahvaltı   59.04
              Yarım Pansiyon   45.12
İzmir         Herşey Dahil     74.70
              Oda + Kahvaltı   41.32
              Yarım Pansiyon   59.61
Name: Price, dtype: float64

In [12]:
bins = [-1, 7, 30, 90, df["SaleCheckInDayDiff"].max()]
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]
df["EB_Score"] = pd.cut(df["SaleCheckInDayDiff"], bins=bins, labels=labels)
df.head(5).to_excel("eb_scorew.xlsx",index=False)

EB_Score, SaleCheckInDayDiff değişkenini belirli aralıklara böler (örneğin, 0-7 gün, 7-30 gün, 30-90 gün, 90+ gün). EB_Score ,belirli bir iş sürecini veya müşteri davranışını ölçmek amacıyla oluşturulmuş bir kategorik değişken veya puanlama sistemini temsil eder.

In [13]:
df.groupby(["SaleCityName", "ConceptName", "EB_Score"], observed=True).agg({"Price": ["mean", "count"]})

Price       
                                                 mean  count
SaleCityName ConceptName    EB_Score                        
Antalya      Herşey Dahil   Last Minuters       62.75  14148
                            Potential Planners  64.90   8874
                            Planners            67.88   4490
                            Early Bookers       66.49   3281
             Oda + Kahvaltı Last Minuters       65.35    503
                            Potential Planners  57.74     75
                            Planners            39.85     15
                            Early Bookers       35.00      5
             Yarım Pansiyon Last Minuters       70.43    204
                            Potential Planners  55.64     39
                            Planners            58.03      9
                            Early Bookers       35.03      4
Aydın        Herşey Dahil   Last Minuters       51.71   3965
                            Potential Planners  52.09   2717
                            Planners            57.88   1834
                            Early Bookers       57.44   2060
             Oda + Kahvaltı Last Minuters       37.80     20
                            Potential Planners  31.08     15
                            Planners            30.20      2
                            Early Bookers       26.81      1
             Yarım Pansiyon Last Minuters       30.87     18
                            Potential Planners  23.73      6
                            Planners            28.81      6
                            Early Bookers       59.60      1
Diğer        Herşey Dahil   Last Minuters       75.47    155
                            Potential Planners  85.79    129
                            Planners            87.26    120
                            Early Bookers       94.79    101
             Oda + Kahvaltı Last Minuters       38.11    539
                            Potential Planners  34.98    176
                            Planners            43.44     34
                            Early Bookers       35.72      8
             Yarım Pansiyon Last Minuters       39.47   1173
                            Potential Planners  45.70    605
                            Planners            46.14    173
                            Early Bookers       50.68     27
Girne        Herşey Dahil   Last Minuters       93.25     24
                            Potential Planners  94.95     41
                            Planners           103.41     25
                            Early Bookers      102.38     16
             Oda + Kahvaltı Last Minuters       37.58     62
                            Potential Planners  39.77     39
                            Planners            43.15     28
                            Early Bookers       43.97     10
             Yarım Pansiyon Last Minuters       60.86     73
                            Potential Planners  53.40     81
                            Planners            45.99     43
                            Early Bookers       33.60     13
Muğla        Herşey Dahil   Last Minuters       63.69   3851
                            Potential Planners  62.41   2942
                            Planners            63.44   1959
                            Early Bookers       61.52   1067
             Oda + Kahvaltı Last Minuters       51.09    331
                            Potential Planners  63.11    149
                            Planners            72.39    121
                            Early Bookers       66.76     53
             Yarım Pansiyon Last Minuters       41.88    104
                            Potential Planners  45.29     50
                            Planners            52.80     24
                            Early Bookers       61.14      9
İzmir        Herşey Dahil   Last Minuters       73.25    589
                            Potential Planners  67.77    440
                            Planners            80.04    231
                      

In [14]:
agg_df=df.groupby(["SaleCityName", "ConceptName", "Seasons"], observed=True).agg({"Price": "mean"}).sort_values("Price",ascending=False)
agg_df.head(20)

Price
SaleCityName ConceptName    Seasons       
Girne        Herşey Dahil   High    103.94
                            Low      90.94
İzmir        Yarım Pansiyon High     87.66
Diğer        Herşey Dahil   Low      87.31
                            High     83.79
İzmir        Herşey Dahil   High     74.75
                            Low      74.31
Antalya      Yarım Pansiyon High     73.26
             Oda + Kahvaltı High     66.27
             Herşey Dahil   High     64.92
Muğla        Herşey Dahil   High     63.38
Antalya      Yarım Pansiyon Low      62.00
             Herşey Dahil   Low      61.55
Muğla        Oda + Kahvaltı High     61.26
Antalya      Oda + Kahvaltı Low      60.67
Girne        Yarım Pansiyon High     58.38
İzmir        Yarım Pansiyon Low      55.35
Aydın        Herşey Dahil   High     54.95
İzmir        Oda + Kahvaltı High     54.14
Girne        Yarım Pansiyon Low      48.58

In [15]:
#indexte yer alan isimleri değişken ismine çevirelim
agg_df.reset_index(inplace=True)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.94
1,Girne,Herşey Dahil,Low,90.94
2,İzmir,Yarım Pansiyon,High,87.66
3,Diğer,Herşey Dahil,Low,87.31
4,Diğer,Herşey Dahil,High,83.79


agg_df DataFrame'inin mevcut indeksini sıfırlar ve onu varsayılan tam sayı indeksine döndürür. Eğer agg_df'nin mevcut bir özel indeksi (örneğin, Date, City gibi sütunlardan oluşturulmuş bir indeks) varsa, bu indeks sıfırlanarak bir sütun haline gelir ve DataFrame'inize yeni bir tam sayı indeksi atanır.
inplace=True: Bu parametre, bu değişikliğin doğrudan agg_df üzerinde yapılmasını sağlar, yani orijinal DataFrame (agg_df) güncellenir ve sonuç agg_df'ye atanmaz. reset_index() metodu None döner, çünkü değişiklik inplace olarak yapılmıştır.

In [16]:
#yeni level_based satışları tanımlayıp verisetine değişken olarak ekliyoruz
agg_df['sales_level_based']=agg_df[["SaleCityName","ConceptName","Seasons"]].agg(lambda x:'_'.join(x).upper(),axis=1)
agg_df

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Girne,Herşey Dahil,High,103.94,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.94,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.66,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.31,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.79,DIĞER_HERŞEY DAHIL_HIGH
5,İzmir,Herşey Dahil,High,74.75,İZMIR_HERŞEY DAHIL_HIGH
6,İzmir,Herşey Dahil,Low,74.31,İZMIR_HERŞEY DAHIL_LOW
7,Antalya,Yarım Pansiyon,High,73.26,ANTALYA_YARIM PANSIYON_HIGH
8,Antalya,Oda + Kahvaltı,High,66.27,ANTALYA_ODA + KAHVALTI_HIGH
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH


In [17]:
agg_df["Segment"]=pd.qcut(agg_df["Price"],4,labels=["D","C","B","A"])
agg_df.head(30)
agg_df.groupby("Segment").agg({"Price":["mean","max","sum"]})

/tmp/ipykernel_18/1129444205.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agg_df.groupby("Segment").agg({"Price":["mean","max","sum"]})


Price              
         mean    max    sum
Segment                    
D       33.37  39.48 300.30
C       44.89  54.14 403.99
B       60.27  64.92 542.47
A       82.47 103.94 742.21

Amaç: Price sütunundaki verileri dört eşit gruba (çeyreklik dilimlere) ayırmak ve her bir gruba bir etiket (segment) atamak.
pd.qcut(): Bu, bir sütundaki verileri belirli sayıda eşit gruba böler. qcut, veri setini çeyrekliklere bölerken her çeyreklik dilimde yaklaşık olarak aynı sayıda örnek bulunur.
Sonuç: agg_df DataFrame'ine Segment adlı yeni bir sütun eklenir. Her satır, Price değerine göre bir segmentte yer alır ve bu segmentin etiketi D, C, B veya A olur.

In [18]:
agg_df.sort_values(by="Price")

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,Segment
35,Aydın,Yarım Pansiyon,Low,25.27,AYDIN_YARIM PANSIYON_LOW,D
34,Aydın,Oda + Kahvaltı,High,30.39,AYDIN_ODA + KAHVALTI_HIGH,D
33,Aydın,Yarım Pansiyon,High,32.63,AYDIN_YARIM PANSIYON_HIGH,D
32,Muğla,Yarım Pansiyon,Low,32.68,MUĞLA_YARIM PANSIYON_LOW,D
31,İzmir,Oda + Kahvaltı,Low,33.56,İZMIR_ODA + KAHVALTI_LOW,D
30,Aydın,Herşey Dahil,Low,33.68,AYDIN_HERŞEY DAHIL_LOW,D
29,Diğer,Oda + Kahvaltı,High,34.84,DIĞER_ODA + KAHVALTI_HIGH,D
28,Muğla,Oda + Kahvaltı,Low,37.77,MUĞLA_ODA + KAHVALTI_LOW,D
27,Girne,Oda + Kahvaltı,High,39.48,GIRNE_ODA + KAHVALTI_HIGH,D
26,Diğer,Yarım Pansiyon,High,39.73,DIĞER_YARIM PANSIYON_HIGH,C


Şimdi De ANTALYA_HERŞEY DAHIL_HIGH hangi segmentte ve ne kadar ücret beklenmektedir onu bulalım

In [19]:
yeni="ANTALYA_HERŞEY DAHIL_HIGH"
agg_df[agg_df["sales_level_based"]==yeni]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,Segment
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH,B
